# Compute Magnitudes Resolution for PWV from Flat SED in LSST filtersFor MERRA2

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2024/10/15
- last update : 2024/10/19 : update pwv reolution repeatability 26 mm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
import scipy
from scipy.optimize import curve_fit,least_squares

In [ ]:
from scipy import interpolate

In [ ]:
machine_name = os.uname().nodename
dm_version = "w_2025_35"
path_rubinsimphot = f"repos/repos_{dm_version}/rubinsimphot/src"
#path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py313")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

In [ ]:
def Get_SED_Pickles():
    seddir = os.path.join(fdir, 'pysynphot', 'pickles')
    seddir_uvi = os.path.join(seddir,"dat_uvi")
    seddir_uvk = os.path.join(seddir,"dat_uvk")
    all_pickles_uvi = sorted(os.listdir(seddir_uvi))
    all_pickles_uvk = sorted(os.listdir(seddir_uvk))
    file_ref = os.path.join(seddir_uvk, "pickles_uk.fits")
    hdul = fits.open(file_ref)
    df_pickle = pd.DataFrame(hdul[1].data)
    NSED = len(df_pickle)

    for index in np.arange(NSED):
        filename = df_pickle.loc[index,"FILENAME"].strip()+".fits"
        fullfilename = os.path.join(seddir_uvk,filename) 
        hdul = fits.open(fullfilename)
        dff = pd.DataFrame(hdul[1].data)

In [ ]:
# reference flux in Jy
F0 = ((0.*u.ABmag).to(u.Jy)).value
F0

In [ ]:
def pdf_lognormal(x,a0,mu,sigma):
    """
    """
    pdf = a0*(np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))/ (x * sigma * np.sqrt(2 * np.pi)))
    return pdf

## Imports dedicated to this work

- import the atmospheric transparency emulator (instead of using libradtran code).
- import rubin sim
- import libPhotometricCorrections : encapsulate uninteresting calculation details

### libradtran Emulator

In [ ]:
from importlib.metadata import version
the_ver = version('getObsAtmo')
print(f"Version of getObsAtmo : {the_ver}")

In [ ]:
from getObsAtmo import ObsAtmo
emul = ObsAtmo("LSST")

In [ ]:
WL = emul.GetWL()

#### Library to fit atmosphere

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit

#### Library that encapsulate calculations for Photometric correction

In [ ]:
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections import *

In [ ]:
def set_photometric_parameters(exptime, nexp, readnoise=None):
    # readnoise = None will use the default (8.8 e/pixel). Readnoise should be in electrons/pixel.
    photParams = PhotometricParameters(exptime=exptime, nexp=nexp, readnoise=readnoise)
    return photParams

In [ ]:
def scale_sed(ref_mag, ref_filter, sed):
    fluxNorm = sed.calc_flux_norm(ref_mag, lsst_std[ref_filter])
    sed.multiply_flux_norm(fluxNorm)
    return sed

In [ ]:
# set default photometric parameters to compute ADU
photoparams = set_photometric_parameters(30, 1 , readnoise=None)

#### library rubin_sim defining LSST parameters, namely for photometric calculations

In [ ]:
from rubinsimphot.phot_utils import Bandpass, Sed
from rubinsimphot.data import get_data_dir

## Configuration

### MERRA2

In [ ]:
ls ..

In [ ]:
filename_m2 = "../2025-09-16-SpectroMerra2/MerradataMerged/Merge_inst1_2d_asm_Nx_M2I1NXASM-2021-2025.csv"
filename_m2b = "../2025-09-16-SpectroMerra2/MerradataMerged/Merge_tavg1_2d_aer_Nx_M2T1NXAER-2021-2025.csv"

In [ ]:
df_m = pd.read_csv(filename_m2)
df_mb = pd.read_csv(filename_m2b)

In [ ]:
TMIN = pd.to_datetime(df_m.time.min())
TMAX = pd.to_datetime(df_m.time.max())

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m")

fig = plt.figure(figsize=(18,5))
gs = GridSpec(1, 2,  width_ratios=[2,1],figure=fig)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
        
leg1=ax1.get_legend()
leg2=ax2.get_legend()


ax1.plot(pd.to_datetime(df_m.Time.values), df_m.TQV.values,c="b",lw=0.5,label="Merra2")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("Precipitable water vapor from Merra2")
ax1.legend()
ax1.set_ylabel("PWV (mm)")
#ax.set_xlim(TMIN,TMAX)

data = np.array(df_m.TQV.values)
mean = np.mean(data)
median = np.median(data)
std = np.std(data)
textstr = "\n".join((f"Expected max-range for PWV : ",
                     f"- average : {mean:.2f} mm",
                     f"- median : {median:.2f} mm",
                     f"- sigma : {std:.2f} mm",     
                    ))
ax1.text(0.05, 0.95, textstr, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)

XMIN = 0.
XMAX = 30.
counts,xedges, _ = ax2.hist(df_m.TQV.values,bins=50,histtype="step",color="b",ls="-",lw=3)
xcenters = (xedges[:-1] + xedges[1:]) / 2

xdata = xcenters
ydata = counts

popt, pcov = curve_fit(pdf_lognormal, xdata, ydata, p0 = [100.,1.,0.5])
sigmas = np.diag(pcov)
xfit = np.linspace(XMIN,XMAX,50)
yfit = pdf_lognormal(xfit,*popt)
ax2.plot(xfit,yfit,"-r")
textstr2 = "\n".join((f"Parameters : ",
                     f"- mu : {popt[1]:.2f}",
                     f"- sigma : {popt[2]:.2f}",     
                    ))
ax2.text(0.35, 0.9, textstr2, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.set_xlabel("PWV (mm)")
ax2.set_title("Fitted log-normal pdf")

#figname =f"{pathfigs}/pwv_allpoints_merra2"+figtype
#fig.savefig(figname)
plt.show()


In [ ]:
am0 = 1.20    # airmass
pwv0 = 5.27  # Precipitable water vapor vertical column depth in mm
oz0 = 300.  # Ozone vertical column depth in Dobson Unit (DU)
ncomp=1     # Number of aerosol components
tau0= 0.0 # Vertical Aerosol depth (VAOD) 
beta0 = 1.2 # Aerosol Angstrom exponent

### Initialisation of Atmospheric corrections

In [ ]:
pc = PhotometricCorrections(am0,pwv0,oz0,tau0,beta0)

### Check standard atmosphere

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
axs.plot(pc.WL,pc.atm_std,'k-')
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Standard atmosphere transmission")

### Check LSST instrument throughput

Photometric Correction package should find the instrumental passband of LSST

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_inst[f].wavelen,pc.bandpass_inst[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Instrument throughput (auxtel)")

### Check LSST standard Filter throughputs

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(6,4))
# loop on filter
for index,f in enumerate(filter_tagnames):
    
    axs.plot(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index]) 
    axs.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=0.2) 
    axs.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")
    
axs.set_xlabel("$\\lambda$ (nm)")
axs.set_title("Total filter throughput (auxtel)")

## Distribution of PWV

https://en.wikipedia.org/wiki/Log-normal_distribution


To produce a log-normal distribution with sample mean $\mu_x$ and sample variance $\sigma_x$, use a log-normal distribution with parameters

$$
\mu = \log \frac{\mu_X^2}{\sqrt{\mu_X^2+\sigma_x^2}}
$$

and

$$
\sigma^2 = \log(1+\frac{\sigma_x^2}{\mu_x^2})
$$

#### Old way

In [ ]:
am = am0
oz = oz0
tau= tau0
beta = beta0
# from repeatability
#sigma_pwv = .31/np.sqrt(2.)
sigma_pwv = .26/np.sqrt(2.)
# Generate a positive distribution of PWV of the desited mam pwv0  and width sigma_pwv
mu = np.log(pwv0**2/np.sqrt(pwv0**2+sigma_pwv**2))
sig = np.sqrt(np.log(1+sigma_pwv**2/pwv0**2))



# The log-normal distribution
all_pwv = np.random.lognormal(mean=mu, sigma=sig,size=1000)
NPWV = len(all_pwv)

#### From Merra2

In [ ]:
mu = popt[1]
sig = popt[2]
# The log-normal distribution
all_pwv = np.random.lognormal(mean=mu, sigma=sig,size=1000)
all_pwv = all_pwv[np.where(np.logical_and(all_pwv>0., all_pwv<20.))[0]] 
NPWV = len(all_pwv)

In [ ]:
mu_x = np.mean(all_pwv)
sig_x = np.std(all_pwv)

In [ ]:
textstr = '\n'.join((
    r'$\mu=%.2f$ mm ' % (mu_x, ),
    r'$\sigma=%.2f$ mm' % (sig_x, )))

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,4))
#ax.hist(data,bins=50,range=(0,20.),facecolor="r")
ax.hist(all_pwv,bins=50,range=(0,20.),facecolor="b")
ax.text(0.65, 0.95, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax.set_title("simulated precipitable water vapor")
ax.set_xlabel("PWV (mm)")

## Generate multi-observations

- for all PWV
- Notice :
     - we keep the same airmass as the standard airmass z=1.3
     - No aerosol to avoid confusion

In [ ]:
print(am,oz,tau,beta)

In [ ]:
all_pwv.shape

In [ ]:
data.shape

In [ ]:
pc.CalculateMultiObs(am,all_pwv,oz,tau,beta)

### PWV variation :  Observed filter and normalized response

In [ ]:
NOBS = len(all_pwv)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig = plt.figure(figsize=(12,5))

# Figure 1
axs=fig.add_subplot(1,2,1)
for index,pwv in enumerate(all_pwv):
  
    atm = pc.coll_atm_nonstd[index]
    
    label = f"pwv={pwv:.1f}" 
    axs.plot(pc.WL,atm,color=all_colors[index],label=label,lw=0.5)
   
axs.plot(pc.WL,pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_title("standard and observed transmission")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])

# Figure 2
axs=fig.add_subplot(1,2,2)

all_linestyles = ['-','--','-.',':','-','--','-.',':','-','--','-.',':','-','--','-.',':']

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


for idx_pwv,pwv in enumerate(all_pwv):
    
    label = f"pwv={pwv:.1f}"
    
    for ifilt,f in enumerate(filter_tagnames):
        
        the_x=pc.WL
        the_y=pc.coll_phiArray_nonstd[idx_pwv][ifilt,:]
       
        
        if ifilt==1:
            axs.plot(the_x,the_y,color=all_colors[idx_pwv],linestyle="-",label=label )
        else:
            axs.plot(the_x,the_y,color=all_colors[idx_pwv],linestyle="-")

axs.set_xlabel("$\lambda$ (nm)")
axs.set_title("Normalized observed transmission")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  


plt.tight_layout()
plt.show()



In [ ]:
NOBS = len(all_pwv)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NOBS)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NOBS), alpha=1)


fig = plt.figure(figsize=(6,5))

# Figure 1
axs=fig.add_subplot(1,1,1)
for index,pwv in enumerate(all_pwv):
    atm_bands = pc.coll_bandpass_total_nonstd[index]    
    label = f"pwv={pwv:.1f}" 
    for f in filter_tagnames: 
        axs.plot(atm_bands[f].wavelen,atm_bands[f].sb,color=all_colors[index],label=label,lw=0.5)

axs.plot(pc.WL,pc.atm_std,color="k",lw=2,label="standard atmosphere")
#axs.legend(bbox_to_anchor=(1.03, 1.0))  
axs.set_xlabel("$\lambda$ (nm)")
axs.set_title(f"standard and observed transmission for airmass {am0}")

ax2 = axs.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax2.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax2.set_yticks([])


plt.tight_layout()
plt.show()


## SED

In [ ]:
# Find the throughputs directory 
#fdir = os.getenv('RUBIN_SIM_DATA_DIR')
fdir = get_data_dir()
if fdir is None:  #environment variable not set
    fdir = os.path.join(os.getenv('HOME'), 'rubin_sim_data')

In [ ]:
the_sed_flat = Sed()
the_sed_flat.set_flat_sed()
the_sed_flat.name = 'flat'
zmag = 20.0
flux_norm = the_sed_flat.calc_flux_norm(zmag, pc.bandpass_total_std['z'])
the_sed_flat.multiply_flux_norm(flux_norm)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
ax.plot(the_sed_flat .wavelen,-2.5*np.log10(the_sed_flat.fnu/F0),"b-",label=the_sed_flat.name)

ax.legend()
#ax.set_ylim(1e-17,1e-14)
#ax.set_xlim(300.,2000.)
ax.set_title("Flat SED $F_\\nu$")
ax.set_ylabel(" Magnitude = $-2.5 \log_{10}(F_\\nu/F_0)$")
ax.set_xlabel("$\\lambda \, (nm)$")
ax.yaxis.set_inverted(True)


ax3 = ax.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax3.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax3.set_yticks([])

In [ ]:
the_sed = the_sed_flat

In [ ]:
fig,(ax,ax2) = plt.subplots(1,2,figsize=(16,6))
ax.plot(the_sed .wavelen,the_sed .flambda,"b-",label=the_sed.name)
ax.legend()
ax.set_ylim(1e-17,1e-15)
ax.set_xlim(300.,1200.)
ax.set_title("Flast SED $F_\lambda$")
ax.set_ylabel("$F_\lambda$")
ax.set_xlabel("$\lambda \, (nm)$")


ax2.plot(the_sed .wavelen,the_sed.fnu,"b-",label=the_sed.name)

ax2.set_yscale("log")
ax2.legend()
ax2.set_ylim(1e-5,1e-4)
ax2.set_xlim(300.,1200.)
ax2.set_title("Flat $F_\\nu$")
ax2.set_ylabel("$F_\\nu$")
ax2.set_xlabel("$\lambda \, (nm)$")

ax3 = ax.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax3.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax3.set_yticks([])
    
ax4 = ax2.twinx()
for ifilt,f in enumerate(filter_tagnames):
    ax4.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[ifilt],alpha=0.1) 
    ax4.set_yticks([])

## Calculate magnitudes AB and Observed Magnitudes for standard and non-standard Magnitudes 

- by construction the AB magnitudes are invariant as the true transmission is considered
- the observed magnitudes provide the resolution as the true transmission-observed transmision is not the expected transmission

In [ ]:
mag_std = {}
adu_std = {}
atm_bands = pc.bandpass_total_std
for index,f in enumerate(filter_tagnames) :
    mag_std[f] = the_sed.calc_mag(atm_bands[f])
    adu_std[f] = -2.5*np.log10(the_sed.calc_adu(atm_bands[f],photoparams))

In [ ]:
mag_std

In [ ]:
adu_std

#### Generate magntudes for non standard  atmosphere

In [ ]:
df = pd.DataFrame(columns = ["pwv","magu","magg","magr","magi","magz","magy",
                                   "aduu","adug","adur","adui","aduz","aduy"])

for idx_pwv,pwv in enumerate(all_pwv):
    mag_nonstd = {}
    adu_nonstd = {}
    atm_bands = pc.coll_bandpass_total_nonstd[idx_pwv] 
    for index,f in enumerate(filter_tagnames) :
        mag_nonstd[f] = the_sed.calc_mag(atm_bands[f])
        adu_nonstd[f] = -2.5*np.log10(the_sed.calc_adu(atm_bands[f],photoparams))
   
    df.loc[idx_pwv] = [pwv, mag_nonstd["u"],mag_nonstd["g"],mag_nonstd["r"],mag_nonstd["i"],mag_nonstd["z"],mag_nonstd["y"],
                       adu_nonstd["u"],adu_nonstd["g"],adu_nonstd["r"],adu_nonstd["i"],adu_nonstd["z"],adu_nonstd["y"]] 

In [ ]:
df = df[["pwv","aduu","adug","adur","adui","aduz","aduy"]]

In [ ]:
df

### Compute difference in mmag between the value and the standard value

In [ ]:
for index,f in enumerate(filter_tagnames) :
    label_in = f'adu{f}'
    label_out =f'd_adu{f}'
    df[label_out] = (df[label_in]- adu_std[f])*1000. 

### Drop absolute mags and keep mag difference

In [ ]:
df = df.drop(labels=["aduu","adug","adur","adui","aduz","aduy"],axis=1)

In [ ]:
df

In [ ]:
df_stat = df.describe()
df_stat 

In [ ]:
NF = len(filter_tagnames)
ncols = 3
nrows = int(np.ceil(NF/ncols))
fig,axes = plt.subplots(nrows,ncols,figsize=(16,10),sharex=True)
axs =axes.flatten()

for idx,ax in enumerate(axs):
    f = filter_tagnames[idx]
    label = f'd_adu{f}'
    data_stat = df[label].describe()
    mean = data_stat["mean"]
    std = data_stat["std"]
    textstr = '\n'.join((
    r'$\mu=%.2f$ mmag' % (mean, ),
    r'$\sigma=%.2f$ mmag' % (std, )))
    df[label].hist(ax=ax,bins=50,facecolor=filter_color[idx])
    ax.set_xlabel("$\Delta m$ (mmag)")
    ax.set_title(f"mag(true)-mag(std) in filt {f}")
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
ymin,ymax = ax.get_ylim()
ax.set_ylim(ymin,ymax*1.3)
plt.tight_layout() 

In [ ]:
df

### Compute relative color difference

In [ ]:
df["d_R-I"] = df["d_adur"] -  df["d_adui"]
df["d_I-Z"] = df["d_adui"] -  df["d_aduz"]
df["d_Z-Y"] = df["d_aduz"] -  df["d_aduy"]

In [ ]:
ncols = 3
nrows = 1

fig,axes = plt.subplots(nrows,ncols,figsize=(16,5),sharex=True)
axs =axes.flatten()
col_labels = ["d_R-I","d_I-Z","d_Z-Y"]
col_tags = ["R-I","I-Z","Z-Y"]
col_colors = ["b","g","r"]
for idx,ax in enumerate(axs):
    xlabel = "$\Delta ($" + col_tags[idx]+ ") (mmag)"
    title = "col(true)-col(std) : $\Delta ($" + col_tags[idx]+ ")"
    label = col_labels[idx] 
    data_stat = df[label].describe()
    mean = data_stat["mean"]
    std = data_stat["std"]
    textstr = '\n'.join((
    r'$\mu=%.2f$ mmag' % (mean, ),
    r'$\sigma=%.2f$ mmag' % (std, )))
    df[label].hist(ax=ax,bins=50,facecolor=col_colors[idx])
    ax.set_xlabel(xlabel)
    ax.set_title(title)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top', bbox=props)
ymin,ymax = ax.get_ylim()
ax.set_ylim(ymin,ymax*1.2)
plt.tight_layout() 